# Use Case 7 Scenario Demonstration

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK


Use Case 7 (UC7) is a cybersecurity scenario set in a generic enterprise organisation, where multiple LAN networks are connected via the 'internet' to represent a corporate WAN. Each network is comprised of routers, switches, computers and servers which green agents use to represent a more real-world accurate network architecture and pattern of life.

Comprising of four major networks; `Home Office (HOME)`, `INTERNET (ISP)`, `REMOTE SITE (REMOTE)` and the larger main site `SOME_TECH`, UC7 is a significant step-up in fidelity from the [smaller network of UC2](./Data-Manipulation-E2E-Demonstration.ipynb). Additionally, two new red agents known as Threat Actor Profiles (TAPs) have been introduced which the blue agent can be trained to defend against. 

Lastly, UC7 is intended to be a generic 'out-of-the-box' configuration that demonstrates the flexibility of PrimAITE rather than a predefined 'challenge' that can solved. Users are encouraged to modify, remove and introduce as much as they wish to create their own unique scenarios. 

_This notebook acts as the starting point for any users unfamiliar with UC7 and will sign post other UC7 relevant notebooks which provide further information._

In [ ]:
!primaite setup

C:\Users\CharlieCrane\primaite\4.0.0-dev\notebooks\example_notebooks\UC7-E2E-Demo.ipynb


2025-03-14 15:51:11,067: Performing the PrimAITE first-time setup...
2025-03-14 15:51:11,068: Building the PrimAITE app directories...
2025-03-14 15:51:11,068: Building primaite_config.yaml...
2025-03-14 15:51:11,068: Rebuilding the demo notebooks...
2025-03-14 15:51:11,096: Reset example notebook: C:\Users\CharlieCrane\primaite\4.0.0-dev\notebooks\example_notebooks\UC7-E2E-Demo.ipynb
2025-03-14 15:51:11,165: Rebuilding the example notebooks...
2025-03-14 15:51:11,183: PrimAITE setup complete!


In [ ]:
import yaml
from prettytable import PrettyTable
from primaite.session.environment import PrimaiteGymEnv
from primaite.game.agent.scripted_agents.random_agent import PeriodicAgent
from primaite.game.agent.interface import ProxyAgent
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.system.services.dns.dns_server import DNSServer
from primaite.simulator.system.software import SoftwareHealthState
from primaite.simulator.file_system.file_system_item_abc import FileSystemItemHealthStatus
from primaite.simulator.system.applications.web_browser import WebBrowser
from primaite.simulator.system.services.service import ServiceOperatingState
from primaite.simulator.system.services.database.database_service import DatabaseService
from primaite.simulator.network.hardware.nodes.network.firewall import Firewall
from pprint import pprint
from primaite.config.load import load, _EXAMPLE_CFG

In [ ]:
use_case_7_config = load(_EXAMPLE_CFG/"uc7_config.yaml")
with open(file=_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['io_settings']['save_agent_logs'] = True # Save agent logs
env = PrimaiteGymEnv(env_config=use_case_7_config)

2025-03-14 15:51:19,583: PrimaiteGymEnv RNG seed = None


## Table of Contents

- [Network Description](#network-description)
    - [Home Office](#network--home-office)
    - [Internet](#network--internet)
    - [Remote Site](#network--remote-site)
    - [ST DMZ](#network--some_tech-dmz)
    - [ST Main Site](#network--some_tech-main-site)
        - [ST Head Office](#network--some_tech-main-site--some_tech-head-office-st_ho)
        - [ST Human Resources](#network--some_tech-main-site--some_tech-human-resources-st_hr)
        - [ST Data](#network--some_tech-main-site--some_tech-data-st_data)
        - [ST Project A](#network--some_tech-main-site--some_tech-project-a-st_proj_a)
        - [ST Project B](#network--some_tech-main-site--some_tech-project-b-st_proj_b)
        - [ST Project C](#network--some_tech-main-site--some_tech-project-c-st_proj_c)
- [Agents](#agent-description)
    - [Green POL](#agents--green-pattern-of-life-pol)
        - [DatabaseService](#agents--green-pattern-of-life-pol--database-service)
        - [web-server](#agents--green-pattern-of-life-pol--web-server)
    - [Red Agent](#agents--red-agent)
        - [TAP001](#agents--red-agent--threat-actor-profile-001-tap001)
        - [TAP003](#agents--red-agent--threat-actor-profile-003-tap003)
    - [Blue Agent](#agents--blue-agent)

## NETWORK DESCRIPTION

<p align="center">
    <a href="./_package_data/uc7/uc7_network_detailed_svg.svg" target="_blank">
        <img src="./_package_data/uc7/uc7_network_detailed_svg.svg" alt="Image">
    </a>  
    
</p>

In [ ]:
uc7_network = env.game.simulation.network
uc7_network.show()

+------------------------------------------------------+
|                        Nodes                         |
+-------------------------+----------+-----------------+
| Node                    | Type     | Operating State |
+-------------------------+----------+-----------------+
| HOME-PUB-RT-DR          | router   | ON              |
| HOME-PUB-SW-AS          | switch   | ON              |
| HOME-PUB-PC-1           | computer | ON              |
| HOME-PUB-PC-2           | computer | ON              |
| HOME-PUB-SRV            | server   | ON              |
| ISP-PUB-RT-BR           | router   | ON              |
| ISP-PUB-SRV-DNS         | server   | ON              |
| REM-PUB-FW              | firewall | ON              |
| REM-PUB-RT-DR           | router   | ON              |
| REM-PUB-SW-AS           | switch   | ON              |
| REM-PUB-PC-1            | computer | ON              |
| REM-PUB-PC-2            | computer | ON              |
| REM-PUB-SRV             | ser


| Default Computer Software |
|------------------|
| DNS Client (Service)      | 
| NTP Client (Service)     | 
| web-browser (Application)     |
| database-client (Application)  |  


| Default Server Software |
|------------------|
| DNS Client (Service)      | 
| NTP Client (Service)     | 

### NETWORK | HOME OFFICE 

The HOME OFFICE (HOME) network simulates a generic home office that consist of the following:

- A main LAN with two computers (`HOME-PUB-PC-1`/`2`) and a server (`HOME-PUB-SRV`) using default configurations.
- A switch (`HOME-PUB-SW-AS`) which connects the above hosts.
- Lastly, a router (`HOME-PUB-RT-DR`) which connects the home office to the wider networks. 


In [ ]:
# Home Office PC 1 (HOME-PUB-PC-1)
home_pub_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-1")
home_pub_pc_1.show()
home_pub_pc_1.software_manager.show()

+---------------------------------------------------------------------------+
|                   HOME-PUB-PC-1 Network Interface Cards                   |
+------+------+-------------------+----------------+-------+---------+------+
| Port | Type | MAC Address       | Address        | Speed | Status  | NMNE |
+------+------+-------------------+----------------+-------+---------+------+
| 1    | NIC  | 13:a8:71:2e:2d:72 | 192.168.1.2/24 | 100.0 | Enabled | {}   |
+------+------+-------------------+----------------+-------+---------+------+
+--------------------------+
| HOME-PUB-PC-1 Open Ports |
+--------------------------+
| Port                     |
+--------------------------+
| 21                       |
| 22                       |
| 53                       |
| 80                       |
| 123                      |
| 219                      |
| 5432                     |
+--------------------------+
+----------------------------------------------------------------------------

In [ ]:
# Home Office Router (HOME-PUB-RT-DR)
home_pub_rt_dr: Router = env.game.simulation.network.get_node_by_hostname("HOME-PUB-RT-DR")
home_pub_rt_dr.show_nic()
home_pub_rt_dr.show_open_ports()
home_pub_rt_dr.acl.show()

+---------------------------------------------------------------------------------------+
|                         HOME-PUB-RT-DR Network Interface Cards                        |
+------+-----------------+-------------------+----------------+-------+----------+------+
| Port | Type            | MAC Address       | Address        | Speed | Status   | NMNE |
+------+-----------------+-------------------+----------------+-------+----------+------+
| 1    | RouterInterface | ab:f3:ac:f5:bd:41 | 192.168.1.1/24 | 100.0 | Enabled  | {}   |
| 2    | RouterInterface | 4b:63:a4:02:f9:65 | 10.1.0.2/30    | 100.0 | Enabled  | {}   |
| 3    | RouterInterface | 72:46:1d:93:f2:2d | 127.0.0.1/8    | 100.0 | Disabled | {}   |
| 4    | RouterInterface | cf:ea:2d:1f:24:0e | 127.0.0.1/8    | 100.0 | Disabled | {}   |
| 5    | RouterInterface | 21:66:ea:ed:d0:fe | 127.0.0.1/8    | 100.0 | Disabled | {}   |
+------+-----------------+-------------------+----------------+-------+----------+------+
+---------

### NETWORK | INTERNET (ISP)

The internet (ISP) network intends to represent the internet. Currently, it's not feasible or possible for PrimAITE to simulate an internet service provider as well as the internet as a whole to a high degree of fidelity. Thus, in UC7, the "internet" refers to the following:

- A router (`ISP-PUB-RT-BR`) which connects the all other UC7 LANS together. 
- A server (`ISP-PUB-SRV-DNS`) which acts as the public DNS server that all PC's use. (Default IP of this host is *8.8.8.8*) 

In [ ]:
isp_pub_rt_br: Router = env.game.simulation.network.get_node_by_hostname("ISP-PUB-RT-BR")
isp_pub_rt_br.show_nic()
isp_pub_rt_br.show_open_ports()
isp_pub_rt_br.acl.show()

+--------------------------------------------------------------------------------------+
|                        ISP-PUB-RT-BR Network Interface Cards                         |
+------+-----------------+-------------------+---------------+-------+----------+------+
| Port | Type            | MAC Address       | Address       | Speed | Status   | NMNE |
+------+-----------------+-------------------+---------------+-------+----------+------+
| 1    | RouterInterface | d7:b4:9c:39:f4:ad | 10.1.0.1/30   | 100.0 | Enabled  | {}   |
| 2    | RouterInterface | bb:7a:21:ce:33:ff | 8.8.8.1/28    | 100.0 | Enabled  | {}   |
| 3    | RouterInterface | 26:21:39:3f:68:f7 | 10.1.10.1/30  | 100.0 | Enabled  | {}   |
| 4    | RouterInterface | 6c:f9:ee:e2:5d:ed | 10.1.100.1/30 | 100.0 | Enabled  | {}   |
| 5    | RouterInterface | 28:b3:d8:14:01:30 | 127.0.0.1/8   | 100.0 | Disabled | {}   |
+------+-----------------+-------------------+---------------+-------+----------+------+
+--------------------

In [ ]:
isp_pub_srv_dns: Server = env.game.simulation.network.get_node_by_hostname("ISP-PUB-SRV-DNS")
isp_pub_srv_dns.show_nic()
isp_pub_srv_dns_server: DNSServer = isp_pub_srv_dns.software_manager.software["dns-server"]
isp_pub_srv_dns_server.show()

+-----------------------------------------------------------------------+
|                ISP-PUB-SRV-DNS Network Interface Cards                |
+------+------+-------------------+------------+-------+---------+------+
| Port | Type | MAC Address       | Address    | Speed | Status  | NMNE |
+------+------+-------------------+------------+-------+---------+------+
| 1    | NIC  | b2:e0:0f:68:3e:e0 | 8.8.8.8/28 | 100.0 | Enabled | {}   |
+------+------+-------------------+------------+-------+---------+------+
+----------------------------------+
| ISP-PUB-SRV-DNS DNS Lookup table |
+----------------+-----------------+
| Domain Name    | IP Address      |
+----------------+-----------------+
| some_tech.com  | 192.168.100.2   |
+----------------+-----------------+


### NETWORK | REMOTE SITE 

The remote site (`REMOTE`) network is similar to the previously introduced `HOME` network but includes a firewall between the `ISP` and the `REMOTE` network:

- A main LAN with two computers (`REM-PUB-PC-1`/`2`) and a server (`REM-PUB-SRV`) using default configurations.
- A switch (`REM-PUB-SW-AS`) which connects the above hosts.
- A router (`HOME-PUB-RT-DR`) which connects the home office to a firewall (`REM-PUB-FW`) which then connects back to the `INTERNET` network.

By default the `REM-PUB-FW` will not block any traffic.

In [ ]:
rem_pub_fw: Firewall = uc7_network.get_node_by_hostname(hostname="REM-PUB-FW")
rem_pub_fw.show_nic()

+----------------------------------------------------------------------------------------+
|                           REM-PUB-FW Network Interface Cards                           |
+------+-----------------+-------------------+-----------------+-------+----------+------+
| Port | Type            | MAC Address       | Address         | Speed | Status   | NMNE |
+------+-----------------+-------------------+-----------------+-------+----------+------+
| 1    | RouterInterface | 64:b1:9d:6c:8d:4a | 10.1.10.2/30    | 100.0 | Enabled  | {}   |
| 2    | RouterInterface | b0:0d:54:f3:9f:22 | 192.168.10.1/30 | 100.0 | Enabled  | {}   |
| 3    | RouterInterface | d9:a6:cf:b1:70:a9 | 127.0.0.1/8     | 100.0 | Disabled | {}   |
+------+-----------------+-------------------+-----------------+-------+----------+------+


In [ ]:
# By default all of the `REM_PUB_FW` acls are configured to permit all traffic
rem_pub_fw.acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                       REM-PUB-FW Access Control List                                      |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | DENY   | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+


### NETWORK | SOME_TECH DMZ (ST DMZ)

The `ST_DMZ` network is a small but important segment which sits between the `INTERNET` network (and thus the `REMOTE` and `HOME` networks as well) and the wider `SOME_TECH` main site networks. Additionally, accessible through the `ST_DMZ` firewall (`ST-PUB-FW`)'s DMZ port is a server which hosts a `web-server` that is accessible to all hosts in the UC7 network.

In [ ]:
# ST DMZ Public Firewall (Permits all traffic by default)
st_pub_fw: Firewall = uc7_network.get_node_by_hostname(hostname="ST_PUB-FW")
st_pub_fw.show_nic()

+----------------------------------------------------------------------------------------+
|                           ST_PUB-FW Network Interface Cards                            |
+------+-----------------+-------------------+------------------+-------+---------+------+
| Port | Type            | MAC Address       | Address          | Speed | Status  | NMNE |
+------+-----------------+-------------------+------------------+-------+---------+------+
| 1    | RouterInterface | 4e:48:cf:7f:ce:04 | 10.1.100.2/30    | 100.0 | Enabled | {}   |
| 2    | RouterInterface | 87:24:56:ab:86:e1 | 192.168.150.1/28 | 100.0 | Enabled | {}   |
| 3    | RouterInterface | d6:f3:5f:da:60:52 | 192.168.100.1/30 | 100.0 | Enabled | {}   |
+------+-----------------+-------------------+------------------+-------+---------+------+


In [ ]:
# ST DMZ Public web-server
st_dmz_pub_srv_web: Server = uc7_network.get_node_by_hostname(hostname="ST_DMZ-PUB-SRV-WEB")
st_dmz_pub_srv_web.software_manager.show()

+---------------------------------------------------------------------------------------+
|                          ST_DMZ-PUB-SRV-WEB Software Manager                          |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

### NETWORK | SOME_TECH MAIN SITE

SOME TECH's main site is comprised of 6 different networks:

| ST Network | Purpose |
|------------|---------|
| `ST_HO` | Some Tech's Head Office. |
| `ST_HR` | Some Tech's Human Resourcing department |
| `ST_DATA` | Contains the ST database and backup server (FTP) |
| `ST_PROJ_A` | Development Network |
| `ST_PROJ_B` | Development Network |
| `ST_PROJ_C` | Development Network |

In order for hosts to communicate between each network and the wider internet, the main site utilises three routers' `ST_INTRA-PRV-RT-DR-1`, `ST_INTRA-PRV-RT-DR-2` and `ST_INTRA-PRV-RT-CR`.

The `ST_INTRA-PRV-RT-DR-1` router is responsible for routing all traffic from the `ST_PROJ_A`/`B`/`C` networks whereas the `ST_INTRA-PRV-RT-DR-2` router routes all traffic from the `ST_HO`/`HR` networks. Both of which then forward all traffic to the main `ST_INTRA-PRV-RT-CR` router. 

This central router connects to the `ST_DMZ` firewall (`ST-PUB-FW`) as well as any traffic that is headed to the `ST_DATA` (the ST database and database storage) network.


In [ ]:
st_intra_prv_rt_cr: Router = uc7_network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_cr.route_table.show()

+---------------------------------------------------+
|           ST_INTRA-PRV-RT-CR Route Table          |
+-------+------------------+---------------+--------+
| Index | Address          | Next Hop      | Metric |
+-------+------------------+---------------+--------+
| 0     | 192.168.200.0/29 | 192.168.170.2 | 0.0    |
| 1     | 192.168.210.0/29 | 192.168.170.2 | 0.0    |
| 2     | 192.168.230.0/29 | 192.168.160.2 | 0.0    |
| 3     | 192.168.240.0/29 | 192.168.160.2 | 0.0    |
| 4     | 192.168.250.0/29 | 192.168.160.2 | 0.0    |
+-------+------------------+---------------+--------+


In [ ]:
st_intra_prv_rt_dr_1: Router = uc7_network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-DR-1")

In [ ]:
st_intra_prv_rt_dr_2: Router = uc7_network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-DR-2")

### NETWORK | SOME_TECH MAIN SITE | SOME_TECH HEAD OFFICE (`ST_HO`)

The some tech head office (`ST_HO`) is a simple LAN containing three computers with the default PC configuration.

In [ ]:
st_head_office_private_pc_1: Computer = uc7_network.get_node_by_hostname("ST_HO-PRV-PC-1")
st_head_office_private_pc_1.software_manager.show()

+---------------------------------------------------------------------------------------+
|                            ST_HO-PRV-PC-1 Software Manager                            |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

### NETWORK | SOME_TECH MAIN SITE | SOME_TECH HUMAN RESOURCES (`ST_HR`)

Similarly, the some tech head human resources office (`ST_HR`) consisting of three default PC configurations computers.

In [ ]:
st_human_resources_private_pc_2: Computer = uc7_network.get_node_by_hostname("ST_HR-PRV-PC-2")
st_human_resources_private_pc_2.software_manager.show()

+---------------------------------------------------------------------------------------+
|                            ST_HR-PRV-PC-2 Software Manager                            |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

### NETWORK | SOME_TECH MAIN SITE | SOME_TECH DATA (`ST_DATA`)

The `ST_DATA` networks contains two servers pivotal to the daily operation of ``SOME_TECH``.

| Server | Purpose |
|--------|---------|
| `ST_DATA-PRV-SRV-DB` | Hosts the `database-service` that all `database-client` are configured to use. | 
| `ST_DATA-PRV-SRV-STORAGE`| Acts as a storage server for the `ST_DATA-PRV-SRV-DB`. |

In [ ]:
st_data_private_server_database: Server = uc7_network.get_node_by_hostname("ST_DATA-PRV-SRV-DB")
st_data_private_server_database_service: DatabaseService = st_data_private_server_database.software_manager.software["database-service"]
st_data_private_server_database.software_manager.show()
st_data_private_server_database.software_manager.file_system.show(full=True)
st_data_private_server_database_service.backup_server_ip

+---------------------------------------------------------------------------------------+
|                          ST_DATA-PRV-SRV-DB Software Manager                          |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

IPv4Address('192.168.220.2')

In [ ]:
st_data_private_server_storage: Server = uc7_network.get_node_by_hostname("ST_DATA-PRV-SRV-STORAGE")
st_data_private_server_storage.software_manager.show()

+---------------------------------------------------------------------------------------+
|                        ST_DATA-PRV-SRV-STORAGE Software Manager                       |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

### NETWORK | SOME_TECH MAIN SITE | SOME_TECH PROJECT A (`ST_PROJ_A`)

All of the `ST_PROJ_A`/`B`/`C` project networks contain three computers and a switch which connects to the `ST_INTRA-PRV-RT-DR-1` router (as described previously).

In [ ]:
st_project_a_private_pc_1: Computer = uc7_network.get_node_by_hostname("ST_PROJ-A-PRV-PC-1")
st_project_a_private_pc_1.software_manager.show()

+---------------------------------------------------------------------------------------+
|                          ST_PROJ-A-PRV-PC-1 Software Manager                          |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

### NETWORK | SOME_TECH MAIN SITE | SOME_TECH PROJECT B (`ST_PROJ_B`)

In [ ]:
st_project_b_private_pc_2: Computer = uc7_network.get_node_by_hostname("ST_PROJ-B-PRV-PC-2")
st_project_b_private_pc_2.software_manager.show()

+---------------------------------------------------------------------------------------+
|                          ST_PROJ-B-PRV-PC-2 Software Manager                          |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

### NETWORK | SOME_TECH MAIN SITE | SOME_TECH PROJECT C (`ST_PROJ_C`)

In [ ]:
st_project_c_private_pc_3: Computer = uc7_network.get_node_by_hostname("ST_PROJ-C-PRV-PC-3")
st_project_c_private_pc_3.software_manager.show()

+---------------------------------------------------------------------------------------+
|                          ST_PROJ-C-PRV-PC-3 Software Manager                          |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

## Agent Description / Demonstration

### AGENTS | Green Pattern of Life (*PoL*)

The UC7 green pattern of life refers to the traffic and behaviour generated by the default `32` different green agents. For each host on the UC7 network, two green agents will attempt to execute the `web-browser` and `database-client` applications and receive a positive or negative reward dependant on whenever they are successful or not.

The table below gives a summary of the different green agent behaviour split between the different UC7 networks.

| **Network** |**Agent Host** | **Application** | **Behaviour Type** | **Probabilities** | **Start Step** | **Start Variance** | **Frequency** | **Variance** | **Reward Impact**  |
|:-----------:|:-------:|:---------------:|:------------------:|:-----------------:|:--------------:|:------------------:|:-------------:|:------------:|:------------------:|
| `HOME SITE` |`HOME-PUB-PC-1`| `database-client`| *Periodic*         |                   |**4**           |**1**               |**4**          |**1**         |**MEDIUM**          |
| `HOME SITE` |`HOME-PUB-PC-2`| `database-client`| *Periodic*         |                   |**8**           |**1**               |**4**          |**1**         |**MEDIUM**          |
| `HOME SITE` |`HOME-PUB-PC-1`/`2`| `web-browser`    | *Probabilistic*    |**20%**            |                |                    |               |              |**LOW**             |
| `REMOTE SITE` |`REM-PUB-PC-1`| `database-client`| *Periodic*         |                   |**12**           |**1**               |**4**          |**1**         |**MEDIUM**          |
| `REMOTE SITE` |`REM-PUB-PC-2`| `database-client`| *Periodic*         |                   |**16**           |**1**               |**4**          |**1**         |**MEDIUM**          |
| `REMOTE SITE` |`REM-PUB-PC-1`/`2`| `web-browser`    | *Probabilistic*    |**20%**            |                |                    |               |              |**LOW**             |
| `ST PROJECT A`/`B`/`C` |`ST_PROJ-*-PRV-PC-1`| `database-client`| *Periodic*         |                   |**1**           |**1**               |**4**          |**1**         |**HIGH**          |
| `ST PROJECT A`/`B`/`C` |`ST_PROJ-*-PRV-PC-1`| `web-browser`    | *Probabilistic*    |**40%**            |                |                    |               |              |**LOW**             |
| `ST PROJECT A`/`B`/`C` |`ST_PROJ-*-PRV-PC-2`/`3`| `database-client`| *Periodic*         |                   |**1**           |**1**               |**4**          |**1**         |**MEDIUM**          |
| `ST PROJECT A`/`B`/`C` |`ST_PROJ-*-PRV-PC-2`/`3`| `web-browser`    | *Probabilistic*    |**20%**            |                |                    |               |              |**LOW**             |
| `ST HEAD OFFICE` |`ST-HO-PRV-PC-1`| `web-browser`    | *Probabilistic*    |**60%**            |                |                    |               |              |**HIGH**             |
| `ST HEAD OFFICE` |`ST-HO-PRV-PC-2`/`3`| `web-browser`    | *Probabilistic*    |**60%**            |                |                    |               |              |**MEDIUM**             |
| `ST HUMAN RESOURCES` |`ST_HR-PRV-PC-1`| `web-browser`    | *Probabilistic*    |**60%**            |                |                    |               |              |**MEDIUM**             |
| `ST HUMAN RESOURCES` |`ST_HR-PRV-PC-2`/`3`| `web-browser`    | *Probabilistic*    |**60%**            |                |                    |               |              |**LOW**             |


For the full details on each green agent then please click on the drop-down menu below:

<details>
    <summary>UC7 Green Agent Full Details</summary>

 **ID** | **PoL Type** | **Description of Activity**                         | **Agent Name**      | **Source Node**    | **Source App / Service** | **Destination Node**    | **Destination App / Service** | **Transport Protocol** | **Application Protocol** | **Behaviour Type** | **Probabilities** | **Start Step** | **Start Variance** | **Max Executions** | **Frequency** | **Variance** | **Reward Impact** 
:------:|:------------:|:---------------------------------------------------:|:-------------------:|:------------------:|:------------------------:|:-----------------------:|:-----------------------------:|:----------------------:|:------------------------:|:------------------:|:-----------------:|:--------------:|:------------------:|:------------------:|:-------------:|:------------:|:-----------------:
 1      | AGENT        | Home Worker accessing Some Tech database            | HOME_WORKER-1       | HOME-PUB-PC-1      | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 4              | 1                  | 1000               | 4             | 1            | MEDIUM            
 2      | AGENT        | Home Worker accessing Some Tech web pages           | HOME_WORKER-1       | HOME-PUB-PC-1      | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 3      | AGENT        | Home Worker accessing Some Tech database            | HOME_WORKER-2       | HOME-PUB-PC-2      | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 8              | 1                  | 1000               | 4             | 1            | MEDIUM            
 4      | AGENT        | Home Worker accessing Some Tech web pages           | HOME_WORKER-2       | HOME-PUB-PC-2      | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 5      | AGENT        | Remote Worker accessing Some Tech database          | REMOTE_WORKER-1     | REM-PUB-PC-1       | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 12             | 1                  | 1000               | 4             | 1            | MEDIUM            
 6      | AGENT        | Remote Worker accessing Some Tech web pages         | REMOTE_WORKER-1     | REM-PUB-PC-1       | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 7      | AGENT        | Remote Worker accessing Some Tech database          | REMOTE_WORKER-2     | REM-PUB-PC-2       | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 16             | 1                  | 1000               | 4             | 1            | MEDIUM            
 8      | AGENT        | Remote Worker accessing Some Tech web pages         | REMOTE_WORKER-2     | REM-PUB-PC-2       | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 9      | AGENT        | Senior Developer accessing Some Tech database       | PROJ_A-SENIOR_DEV   | ST_PROJ_A-PRV-PC-1 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | HIGH              
 10     | AGENT        | Senior Developer accessing Some Tech web pages      | PROJ_A-SENIOR_DEV   | ST_PROJ_A-PRV-PC-1 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 40% chance        |                |                    |                    |               |              | LOW               
 11     | AGENT        | Junior Developer accessing Some Tech database       | PROJ_A-JUNIOR_DEV-1 | ST_PROJ_A-PRV-PC-2 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | MEDIUM            
 12     | AGENT        | Junior Developer accessing Some Tech web pages      | PROJ_A-JUNIOR_DEV-1 | ST_PROJ_A-PRV-PC-2 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 13     | AGENT        | Junior Developer accessing Some Tech database       | PROJ_A-JUNIOR_DEV-2 | ST_PROJ_A-PRV-PC-3 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | MEDIUM            
 14     | AGENT        | Junior Developer accessing Some Tech web pages      | PROJ_A-JUNIOR_DEV-2 | ST_PROJ_A-PRV-PC-3 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 15     | AGENT        | Senior Developer accessing Some Tech database       | PROJ_B-SENIOR_DEV   | ST_PROJ_B-PRV-PC-1 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | HIGH              
 16     | AGENT        | Senior Developer accessing Some Tech web pages      | PROJ_B-SENIOR_DEV   | ST_PROJ_B-PRV-PC-1 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 40% chance        |                |                    |                    |               |              | LOW               
 17     | AGENT        | Junior Developer accessing Some Tech database       | PROJ_B-JUNIOR_DEV-1 | ST_PROJ_B-PRV-PC-2 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | MEDIUM            
 18     | AGENT        | Junior Developer accessing Some Tech web pages      | PROJ_B-JUNIOR_DEV-1 | ST_PROJ_B-PRV-PC-2 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 19     | AGENT        | Junior Developer accessing Some Tech database       | PROJ_B-JUNIOR_DEV-2 | ST_PROJ_B-PRV-PC-3 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | MEDIUM            
 20     | AGENT        | Junior Developer accessing Some Tech web pages      | PROJ_B-JUNIOR_DEV-2 | ST_PROJ_B-PRV-PC-3 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 21     | AGENT        | Senior Developer accessing Some Tech database       | PROJ_C-SENIOR_DEV   | ST_PROJ_C-PRV-PC-1 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | HIGH              
 22     | AGENT        | Senior Developer accessing Some Tech web pages      | PROJ_C-SENIOR_DEV   | ST_PROJ_C-PRV-PC-1 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 40% chance        |                |                    |                    |               |              | LOW               
 23     | AGENT        | Junior Developer accessing Some Tech database       | PROJ_C-JUNIOR_DEV-1 | ST_PROJ_C-PRV-PC-2 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | MEDIUM            
 24     | AGENT        | Junior Developer accessing Some Tech web pages      | PROJ_C-JUNIOR_DEV-1 | ST_PROJ_C-PRV-PC-2 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 25     | AGENT        | Junior Developer accessing Some Tech database       | PROJ_C-JUNIOR_DEV-2 | ST_PROJ_C-PRV-PC-3 | `database-client`          | ST_DATA-PRV-SRV-DB      | `database-service`              | TCP                    | PostgreSQL               | PERIODIC           |                   | 1              | 1                  | 1000               | 4             | 1            | MEDIUM            
 26     | AGENT        | Junior Developer accessing Some Tech web pages      | PROJ_C-JUNIOR_DEV-2 | ST_PROJ_C-PRV-PC-3 | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 20% chance        |                |                    |                    |               |              | LOW               
 27     | AGENT        | CEO accessing Some Tech web pages                   | CEO                 | ST_HO-PRV-PC-1     | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 60% chance        |                |                    |                    |               |              | HIGH              
 28     | AGENT        | CTO accessing Some Tech web pages                   | CTO                 | ST_HO-PRV-PC-2     | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 60% chance        |                |                    |                    |               |              | MEDIUM            
 29     | AGENT        | CFO accessing Some Tech web pages                   | CFO                 | ST_HO-PRV-PC-3     | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 60% chance        |                |                    |                    |               |              | MEDIUM            
 30     | AGENT        | Senior HR accessing Some Tech web pages             | SENIOR_HR           | ST_HR-PRV-PC-1     | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 60% chance        |                |                    |                    |               |              | MEDIUM            
 31     | AGENT        | Junior HR accessing Some Tech web pages             | JUNIOR_HR-1         | ST_HR-PRV-PC-2     | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | HTTPS                    | PROBABILISTIC      | 60% chance        |                |                    |                    |               |              | LOW               
 32     | AGENT        | Junior HR accessing Some Tech web pages             | JUNIOR_HR-2         | ST_HR-PRV-PC-3     | `web-browser`              | ST_DMZ-PUB-SRV-WEB      | `web-server`                    | TCP                    | 
</details>

#### AGENTS | Green *PoL* | database-client Agents Demo

The `database-client` green agents will attempt to use their host's `database-client` application to make a simple connection to the `database-service` on the `ST_DATA-PRV-SRV-DB` host (these connections have no direct impact to the `database-service` or the `database.db` file itself).

Additionally, `database-client` green agents are *Periodic* meaning they will attempt to use the database based on game time-steps. Specifically, these agents will begin on the time-step given in their `start_step` setting and will then will reattempt on each subsequence timestep based on the `Frequency` setting. These settings are then randomised using the remaining `start_variance` and `variance` options (also given in timesteps). These values are used to *±* their respective base settings to ensure the green agents achieve a moderate amount of domain randomisation in each PrimAITE episode.

For example, take a *Periodic* green agent set with a `start_step` of **4** and a `frequency` of **4** with a `start_variance` of **1** and a `variance` of **1** will cause a green agent to make its first action on timestep $4 \pm 1$ and then any subsequent actions every $4 \pm 1$ timesteps afterwards.


In [ ]:
env.reset() # Resetting the simulation
home_pub_pc_1_database_green_agent = env.game.agents.get("HOME_WORKER-1-DB")
for time_step in range(10):
    env.step(0)
    if not home_pub_pc_1_database_green_agent.history[time_step].action == 'DONOTHING':
        print(home_pub_pc_1_database_green_agent.history[time_step])


2025-03-14 15:51:20,234: Resetting environment, episode 0, avg. reward: 0.0


timestep=0 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={'connection_attempt_status': 'n/a'} observation=0
timestep=1 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={'connection_attempt_status': 'n/a'} observation=0
timestep=2 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={'connection_attempt_status': 'n/a'} observation=0
timestep=3 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={'connection_attempt_status': 'n/a'} observation=0
timestep=4 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={'connection_attempt_status': 'n/a'} observation=0
timestep=5 action='node-applic

In [ ]:
home_pub_pc_1.software_manager.software["database-client"].sys_log.show(last_n=10)
st_data_private_server_database.software_manager.software["database-service"].sys_log.show(last_n=5)

+-----------------------------+
|    HOME-PUB-PC-1 Sys Log    |
+-----------+-------+---------+
| Timestamp | Level | Message |
+-----------+-------+---------+
+-----------+-------+---------+
+-----------------------------+
|  ST_DATA-PRV-SRV-DB Sys Log |
+-----------+-------+---------+
| Timestamp | Level | Message |
+-----------+-------+---------+
+-----------+-------+---------+


#### AGENTS | Green PoL | `web-browser` Agents Demo

Unlike the `database-client` green agents, the `web-browser` green agents are *probabilistic*. These agents are quite simple; on every timestep a probability roll is made to determine whenever the agent acts. On a successful outcome the agent will attempt to execute the `web-browser` application which will then attempt to connect to the `ST-DMZ-PUB-SRV-WEB` host. On a unsuccessful outcome then the green agent will simply perform not action on this timestep.

For example, a `web-browser` green agent with a `20%` chance has a $\frac{1}{5}$ chance of actioning its host's `web-browser` to access the `ST-DMZ-PUB-SRV-WEB` web-server. 

In [ ]:
env.reset() # Resetting the simulation
home_pub_pc_1_web_browser_green_agent = env.game.agents.get("HOME_WORKER-1-WEB")
for time_step in range(10):
    env.step(0)
    if not home_pub_pc_1_web_browser_green_agent.history[time_step].action == 'DONOTHING':
        print(home_pub_pc_1_web_browser_green_agent.history[time_step])


2025-03-14 15:51:22,320: Resetting environment, episode 1, avg. reward: 12.410937500000006


timestep=0 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={} observation=0
timestep=1 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={} observation=0
timestep=2 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={} observation=0
timestep=3 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={} observation=0
timestep=4 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={} observation=0
timestep=5 action='do-nothing' parameters={} request=['do-nothing'] response=RequestResponse(status='success', data={}) reward=0.0 reward_info={} observation=0
timestep=6 action='do-nothing' parameter

In [ ]:
home_pub_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-1")
home_pub_pc_1_web_browser_green_agent.logger.show()
home_pub_pc_1_web_browser: WebBrowser = home_pub_pc_1.software_manager.software["web-browser"]
home_pub_pc_1_web_browser.sys_log.show()

+---------------------------------+
| HOME_WORKER-1-WEB Behaviour Log |
+------------+-------+------------+
| Time Step  | Level | Message    |
+------------+-------+------------+
+------------+-------+------------+
+-----------------------------+
|    HOME-PUB-PC-1 Sys Log    |
+-----------+-------+---------+
| Timestamp | Level | Message |
+-----------+-------+---------+
+-----------+-------+---------+


### AGENTS | Red Agents

For UC7, two new red agents have been developed which introduce much more complex and realistic attacks in comparison to UC2's [data manipulation red agent](./Data-Manipulation-Customising-Red-Agent.ipynb) for the blue agent to defend against. These new red agents, or more commonly referred to `Threat Actor Profiles` (*TAPS*), utilise a series of different green, blue and red actions to simulate the different steps of a real-world attack.

This notebook does not cover the red agents in much detail, hence its highly recommended that readers should check out the respective TAP notebooks for a much more in-depth look at each TAP and their impacts.


### AGENTS | RED AGENT | Threat Actor Profile 001 (`TAP001`)

This TAP aims to exfiltrate and then encrypt the `database.db` file on `ST_DATA-PRV-SRV-DB` host, whilst leaving the functionality of the database intact. Configured by default to start on the `ST_PROJ-A-PRV-PC-1` host, `TAP001` must first embed itself on the host, locate the target (`ST_DATA-PRV-SRV-DB`) through a series of [`nmap`](/PrimAITE/docs/source/simulation_components/system/applications/nmap.rst) scans, establish a connection to its [`c2-server`](./Command-and-Control-E2E-Demonstration.ipynb)(`ISP-PUB-SRV-DNS` by default) and then finally attempt to exfiltrate and encrypt. 

If successful, the blue agent is configured to receive a serve negative reward and thus must prevent `TAP001` from ever reaching the target database. This could be through blocking its connection to the target or its `c2-server` via a carefully crafted ACL or perhaps through more a forceful approach such as shutting down the starting host.

For more information on `TAP001` and its impacts, [please refer to the TAP001 E2E notebook](./UC7-TAP001-Kill-Chain-E2E.ipynb) or for more blue agent involved demonstration refer to the [UC7 attack variants notebook](./UC7-attack-variants.ipynb) 

In [ ]:
# By default the `uc7_config.yaml` is setup to use TAP001
env.reset()
for _ in range(80):
    env.step(action=0)

uc7_tap001 = env.game.agents.get("attacker")
uc7_tap001.logger.show()

2025-03-14 15:51:23,476: Resetting environment, episode 2, avg. reward: 12.396875000000001


+-----------------------------+
|    attacker Behaviour Log   |
+-----------+-------+---------+
| Time Step | Level | Message |
+-----------+-------+---------+
+-----------+-------+---------+


In [ ]:
# TAP001 starting host
st_project_a_private_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("ST_PROJ-A-PRV-PC-1")
st_project_a_private_pc_1.software_manager.show()

+---------------------------------------------------------------------------------------+
|                          ST_PROJ-A-PRV-PC-1 Software Manager                          |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

In [ ]:
st_project_a_private_pc_1.file_system.show(full=True)
isp_pub_srv_dns: Server = env.game.simulation.network.get_node_by_hostname(hostname="ISP-PUB-SRV-DNS")
isp_pub_srv_dns.file_system.show(full=True)

+----------------------------------------------------------------------------------------+
|                             ST_PROJ-A-PRV-PC-1 File System                             |
+-------------------------------+------+---------------+-----------------------+---------+
| File Path                     | Size | Health status | Visible health status | Deleted |
+-------------------------------+------+---------------+-----------------------+---------+
| downloads/malware_dropper.ps1 | 0 B  | GOOD          | NONE                  | False   |
| root                          | 0 B  | GOOD          | NONE                  | False   |
+-------------------------------+------+---------------+-----------------------+---------+
+--------------------------------------------------------------------+
|                    ISP-PUB-SRV-DNS File System                     |
+-----------+------+---------------+-----------------------+---------+
| File Path | Size | Health status | Visible health status 

In [ ]:
# Database Impact 
st_data_private_server_database: Server = env.game.simulation.network.get_node_by_hostname(hostname="ST_DATA-PRV-SRV-DB")
st_data_private_server_database.file_system.show(full=True)

+----------------------------------------------------------------------------------+
|                          ST_DATA-PRV-SRV-DB File System                          |
+----------------------+---------+---------------+-----------------------+---------+
| File Path            | Size    | Health status | Visible health status | Deleted |
+----------------------+---------+---------------+-----------------------+---------+
| database/database.db | 4.77 MB | GOOD          | NONE                  | False   |
| root                 | 0 B     | GOOD          | NONE                  | False   |
+----------------------+---------+---------------+-----------------------+---------+


### AGENTS | RED AGENT | Threat Actor Profile 003 (`TAP003`)

Unlike `TAP001`'s more traditional representation of a threat actor, `TAP003` represents a malicious insider which leverages its pre-existing knowledge to covertly add malicious access control lists (ACLs) to three different routers each of which affecting green agent traffic in a different way causing the blue agent to receive negative rewards. Thus, the blue agent must learn to leverage its ability to remove rules and change credentials throughout the network to rectify the impacts of `TA003` and re-establish green POL and prevent `TAP003` from accessing additional routers.

The table below is a brief summary of the malicious ACLs added by `TAP003`

|Target Router         | Impact |
|----------------------|--------|
|`ST_INTRA-PRV-RT-DR-1`| Blocks all `POSTGRES_SERVER` that arrives at the `ST_INTRA-PRV-RT-DR-1` router. This rule will prevent all ST_PROJ_* hosts from accessing the database (`ST_DATA-PRV-SRV-DB`).|
|`ST_INTRA-PRV-RT-CR`| Blocks all `HTTP` traffic that arrives at the`ST_INTRA-PRV-RT-CR` router. This rule will prevent all SOME_TECH hosts from accessing the web-server (`ST-DMZ-PUB-SRV-WEB`)|
|`REM-PUB-RT-DR`| Blocks all `DNS` traffic that arrives at the `REM-PUB-RT-DR` router. This rule prevents any remote site works from accessing the DNS Server (`ISP-PUB-SRV-DNS`).|

Lastly, its highly recommended that users refer to the [TAP003 E2E notebook](./UC7-TAP003-Kill-Chain-E2E.ipynb) for further information or for the [UC7 attack variants notebook](./UC7-attack-variants.ipynb) demonstration of TAP003 defence.

In [ ]:
# Loading up the TAP003 UC7 config variant
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg["agents"][33]["agent_settings"]["flatten_obs"] = False
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['io_settings']['save_agent_logs'] = True # Saving agent logs

env = PrimaiteGymEnv(env_config=cfg)

2025-03-14 15:51:34,604: PrimaiteGymEnv RNG seed = None


In [ ]:
# By default the `uc7_config.yaml` is setup to use TAP001
env.reset()
for _ in range(128):
    env.step(action=0)

uc7_tap003 = env.game.agents.get("attacker")
uc7_tap003.logger.show()

2025-03-14 15:51:34,632: Resetting environment, episode 0, avg. reward: 0.0


+----------------------------------------------------------------------------------+
|                              attacker Behaviour Log                              |
+-----------+-------+--------------------------------------------------------------+
| Time Step | Level | Message                                                      |
+-----------+-------+--------------------------------------------------------------+
| 28        | INFO  | Manipulation complete. Progressing to exploit...             |
| 31        | INFO  | Logging into ST_INTRA-PRV-RT-DR-1 in order to add ACL rules. |
| 34        | INFO  | Adding ACL rule to ST_INTRA-PRV-RT-DR-1                      |
| 37        | INFO  | Logging into ST_INTRA-PRV-RT-CR in order to add ACL rules.   |
| 40        | INFO  | Adding ACL rule to ST_INTRA-PRV-RT-CR                        |
| 43        | INFO  | Logging into REM-PUB-RT-DR in order to add ACL rules.        |
| 46        | INFO  | Adding ACL rule to REM-PUB-RT-DR           

In [ ]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1").acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                  ST_INTRA-PRV-RT-DR-1 Access Control List                                 |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 1     | DENY   | tcp      | ANY    | 0.0.255.255  | 5432     | ANY    | 0.0.255.255  | 5432     | 214     |
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 625     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | 

In [ ]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR").acl.show()

+-----------------------------------------------------------------------------------------------------------+
|                                   ST_INTRA-PRV-RT-CR Access Control List                                  |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 1     | DENY   | tcp      | ANY    | 0.0.255.255  | 80       | ANY    | 0.0.255.255  | 80       | 526     |
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 967     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | 

In [ ]:
env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR").acl.show()    

+-----------------------------------------------------------------------------------------------------------+
|                                     REM-PUB-RT-DR Access Control List                                     |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| Index | Action | Protocol | Src IP | Src Wildcard | Src Port | Dst IP | Dst Wildcard | Dst Port | Matched |
+-------+--------+----------+--------+--------------+----------+--------+--------------+----------+---------+
| 1     | DENY   | tcp      | ANY    | 0.0.255.255  | 53       | ANY    | 0.0.255.255  | 53       | 0       |
| 5     | PERMIT | ANY      | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 253     |
| 22    | PERMIT | ANY      | ANY    | ANY          | 219      | ANY    | ANY          | 219      | 0       |
| 23    | PERMIT | icmp     | ANY    | ANY          | ANY      | ANY    | ANY          | ANY      | 0       |
| 24    | 

## AGENTS | Blue Agent




In PrimAITE blue agent configuration is split into three separate sections, each responsible for a different part of the blue agent functionality.

- `observation_space`

The observation space (or more commonly `OBS`) refers to the what simulation components the blue agent observes each `time_step`. A blue agent `OBS` can be configured to be as large the entire network or just an individual node. 

- `action_space`

The action space configuration is used to set the actions available to the blue agent. The larger range of actions could be optimal for responding to a large set of different scenarios but could lead to longer training times. Crafting the perfect `action_space` is pivotal for creating an effective blue agent.

- `reward_function`

The `reward_function` section configures what type of reward (or penalisation) the blue agent will receive based on the current status of the simulation. A balanced and well crafted reward function is the path of success for any blue agent to learn what the best approach to scenario may be.



_The remaining section of this notebook will cover the UC7 blue agent and the default aforementioned settings._

In [ ]:
env.reset() # Resetting the env
defender = env.game.agents.get("defender")

2025-03-14 15:51:41,131: Resetting environment, episode 1, avg. reward: 116.84999999999965


## AGENTS | Blue Agent | Observation Space (OBS)

## NETWORK DESCRIPTION

<p align="center">
    <a href="./_package_data/uc7/blue_agent_action_and_obs.png" target="_blank">
        <img src="./_package_data/uc7/blue_agent_action_and_obs.png" alt="Image">
    </a>  
    
</p>

_(Click to enlarge)_

Represented as a hierarchy (i.e the same way as a dictionary) the blue agent OBS is separate into multiple sections.

### Links

The first section is Links, which is used to report the current network traffic load on every link in the network. Since the observation space is composed of categorical variables, the traffic values are converted from MBit/s to a category based on a percentage of the link's capacity.

<details> <summary> Link Values Mapping</summary>

|load observation|percent utilisation|
|--|--|
|0|exactly 0%|
|1|0-11%|
|2|11-22%|
|3|22-33%|
|4|33-44%|
|5|44-55%|
|6|55-66%|
|7|66-77%|
|8|77-88%|
|9|88-99%|
|10|exactly 100%|


</details>

<details>
    <summary>Link List</summary>
    
|Link number|Endpoint A|Endpoint B|
|---|---|---|
|1|HOME-PUB-SW-AS:eth-1|HOME-PUB-RT-DR:eth-1 |
|2|HOME-PUB-SW-AS:eth-1|HOME-PUB-PC-1:eth-1 |
|3|HOME-PUB-SW-AS:eth-1|HOME-PUB-PC-2:eth-1 |
|4|HOME-PUB-SW-AS:eth-1|HOME-PUB-SRV:eth-1 |            
|5|ISP-PUB-RT-BR:eth-1|HOME-PUB-RT-DR:eth-2 |
|6|ISP-PUB-RT-BR:eth-2|ISP-PUB-SRV-DNS:eth-1 |
|7|ISP-PUB-RT-BR:eth-3|REM-PUB-FW:eth-1 |
|8|REM-PUB-FW:eth-2|REM-PUB-RT-DR:eth-1 |
|9|REM-PUB-RT-DR:eth-2|REM-PUB-SW-AS:eth-1 |
|10|REM-PUB-SW-AS:eth-2|REM-PUB-PC-1:eth-1 |
|11|REM-PUB-SW-AS:eth-3|REM-PUB-PC-2:eth-1 |     
|12|REM-PUB-SW-AS:eth-4|REM-PUB-SRV:eth-1 |
|13|ISP-PUB-RT-BR:eth-4|ST_PUB-FW:eth-1 |
|14|ST_PUB-FW:eth-3|ST_DMZ-PUB-SRV-WEB:eth-1 |
|15|ST_INTRA-PRV-RT-CR:eth-1|ST_PUB-FW:eth-2 |
|16|ST_INTRA-PRV-RT-CR:eth-2|ST_INTRA-PRV-RT-DR-1:eth-1 |
|17|ST_INTRA-PRV-RT-CR:eth-3|ST_INTRA-PRV-RT-DR-2:eth-1 |    
|18|ST_INTRA-PRV-RT-CR:eth-4|ST_DATA-PRV-SW-AS:eth-1 |
|19|ST_HO-PRV-SW-AS:eth-1|ST_INTRA-PRV-RT-DR-2:eth-2 |
|20|ST_HO-PRV-SW-AS:eth-2|ST_HO-PRV-PC-1:eth-1 |
|21|ST_HO-PRV-SW-AS:eth-3|ST_HO-PRV-PC-2:eth-1 |
|22|ST_HO-PRV-SW-AS:eth-4|ST_HO-PRV-PC-3:eth-1 |
|23|ST_HR-PRV-SW-AS:eth-1|ST_INTRA-PRV-RT-DR-2:eth-3 |
|24|ST_HR-PRV-SW-AS:eth-2|ST_HR-PRV-PC-1:eth-1 |
|25|ST_HR-PRV-SW-AS:eth-3|ST_HR-PRV-PC-2:eth-1 |
|26|ST_HR-PRV-SW-AS:eth-4|ST_HR-PRV-PC-3:eth-1 |
|27|ST_DATA-PRV-SW-AS:eth-2|ST_DATA-PRV-SRV-STORAGE:eth-1 |
|28|ST_DATA-PRV-SW-AS:eth-3|ST_DATA-PRV-SRV-DB:eth-1 |  
|29|ST_INTRA-PRV-RT-DR-1:eth-2|ST_PROJ-A-PRV-SW-AS:eth-1 |
|30|ST_PROJ-A-PRV-SW-AS:eth2|ST_PROJ-A-PRV-PC-1:eth-1|
|31|ST_PROJ-A-PRV-SW-AS:eth3|ST_PROJ-A-PRV-PC-2:eth-1 |
|32|ST_PROJ-A-PRV-SW-AS:eth4|ST_PROJ-A-PRV-PC-3:eth-1 |        
|33|ST_INTRA-PRV-RT-DR-1:eth-3|ST_PROJ-B-PRV-SW-AS:eth-1 |
|34|ST_PROJ-B-PRV-SW-AS:eth2|ST_PROJ-B-PRV-PC-1:eth-1 |
|35|ST_PROJ-B-PRV-SW-AS:eth3|ST_PROJ-B-PRV-PC-2:eth-1 |
|36|ST_PROJ-B-PRV-SW-AS:eth4|ST_PROJ-B-PRV-PC-3:eth-1 | 
|37|ST_INTRA-PRV-RT-DR-1:eth-4|ST_PROJ-C-PRV-SW-AS:eth-1 |
|38|ST_PROJ-A-PRV-SW-AS:eth2|ST_PROJ-C-PRV-PC-1:eth-1 |
|39|ST_PROJ-A-PRV-SW-AS:eth3|ST_PROJ-C-PRV-PC-2:eth-1 |
|40|ST_PROJ-A-PRV-SW-AS:eth4|ST_PROJ-C-PRV-PC-3:eth-1 |

</details>

The code cell below prints out how the above link list table appears in the blue agent observation space

In [ ]:
env.reset()

obs, reward, _,_,info = env.step(0)
for i,x in obs['LINKS'].items():
    print(i, x)


2025-03-14 15:51:41,892: Resetting environment, episode 2, avg. reward: 0.0


1 {'PROTOCOLS': {'ALL': 1}}
2 {'PROTOCOLS': {'ALL': 0}}
3 {'PROTOCOLS': {'ALL': 0}}
4 {'PROTOCOLS': {'ALL': 0}}
5 {'PROTOCOLS': {'ALL': 1}}
6 {'PROTOCOLS': {'ALL': 1}}
7 {'PROTOCOLS': {'ALL': 0}}
8 {'PROTOCOLS': {'ALL': 0}}
9 {'PROTOCOLS': {'ALL': 0}}
10 {'PROTOCOLS': {'ALL': 0}}
11 {'PROTOCOLS': {'ALL': 0}}
12 {'PROTOCOLS': {'ALL': 0}}
13 {'PROTOCOLS': {'ALL': 1}}
14 {'PROTOCOLS': {'ALL': 1}}
15 {'PROTOCOLS': {'ALL': 1}}
16 {'PROTOCOLS': {'ALL': 1}}
17 {'PROTOCOLS': {'ALL': 1}}
18 {'PROTOCOLS': {'ALL': 1}}
19 {'PROTOCOLS': {'ALL': 1}}
20 {'PROTOCOLS': {'ALL': 1}}
21 {'PROTOCOLS': {'ALL': 1}}
22 {'PROTOCOLS': {'ALL': 1}}
23 {'PROTOCOLS': {'ALL': 1}}
24 {'PROTOCOLS': {'ALL': 0}}
25 {'PROTOCOLS': {'ALL': 1}}
26 {'PROTOCOLS': {'ALL': 1}}
27 {'PROTOCOLS': {'ALL': 4}}
28 {'PROTOCOLS': {'ALL': 4}}
29 {'PROTOCOLS': {'ALL': 1}}
30 {'PROTOCOLS': {'ALL': 0}}
31 {'PROTOCOLS': {'ALL': 0}}
32 {'PROTOCOLS': {'ALL': 0}}
33 {'PROTOCOLS': {'ALL': 1}}
34 {'PROTOCOLS': {'ALL': 0}}
35 {'PROTOCOLS': {'ALL'

#### HOSTS

By default the blue agent is monitoring **3** different computers and **4** server:

|Host label|hostname| services | applications | folders | files |
|:--:|:--:|:--:|:--:|:--:|:--:|
|HOST0|**ST_PROJ-A-PRV-PC-1** | `ftp-client` | `ransomware_script`, `database-client` | `downloads`, `exfiltration_folder `|` malware_dropper.ps1`, `database.db` |
|HOST1|**ST_PROJ-B-PRV-PC-2**|  `ftp-client` | `ransomware-script`, `database-client` | `downloads`, `exfiltration_folder` | `malware_dropper.ps1`, `database.db` |
|HOST2|**ST_PROJ-C-PRV-PC-3**|  `ftp-client` | `ransomware-script`, `database-client `|` downloads`, `exfiltration_folder` | `malware_dropper.ps1`, `database.db` |
|HOST3|**ST_DATA-PRV-SRV-DB**||| `database` | `database.db`|


Each `time_step` these hosts report the following to the blue agent:

- operating status 
- number of file creations
- number of file deletions
- up to 4 installed services (operating status and health status)
- up to 2 installed applications (operating status, health status, and number of executions)
- up to 1 folder (health status)
- up to 1 file on the folders (health status and number of accesses)
- 1 network interface, including: 
  - operating status
  - number of malicious network events detected
  - amount of incoming and outgoing traffic on each protocol and port



It's worth noting that for larger observation spaces, not every host will have the maximum number of files, folders, services, and applications configured for the observation space, so padding is used to ensure that the shape of each host observation is the same. The hosts appear in the order that they are defined in the config file.

In [ ]:
env.reset()

obs, reward, _,_,info = env.step(0)
for node_id, node_obs in obs['NODES'].items():
    if not "ROUTER" in node_id: # filter out router OBS for now
        print(node_id)
        pprint(node_obs)


2025-03-14 15:51:42,606: Resetting environment, episode 3, avg. reward: 1.1140625000000002


HOST0
{'APPLICATIONS': {1: {'health_status': 0,
                      'num_executions': 0,
                      'operating_status': 0},
                  2: {'health_status': 0,
                      'num_executions': 0,
                      'operating_status': 1}},
 'FOLDERS': {1: {'FILES': {1: {'health_status': 0, 'num_access': 0}},
                 'health_status': 0},
             2: {'FILES': {1: {'health_status': 0, 'num_access': 0}},
                 'health_status': 0}},
 'NICS': {1: {'NMNE': {'inbound': 0, 'outbound': 0},
              'TRAFFIC': {'icmp': {'inbound': 0, 'outbound': 0},
                          'tcp': {80: {'inbound': 0, 'outbound': 0},
                                  5432: {'inbound': 0, 'outbound': 0}}},
              'nic_status': 1}},
 'SERVICES': {1: {'health_status': 0, 'operating_status': 2},
              2: {'health_status': 0, 'operating_status': 0}},
 'num_file_creations': 0,
 'num_file_deletions': 0,
 'operating_status': 1,
 'users': {'local_lo

For any readers unfamiliar with the different enumeration values in any of the obs output (i.e what the `0`,`1`,`2`,`3` values represent) then please refer to the following tables for reference:

<details> <summary> Host operating state category table</summary>

|value|meaning|
|-|-|
|0|UNUSED|
|1|ON|
|2|OFF|
|3|BOOTING|
|4|SHUTTING DOWN|

</details>

<details> <summary> No. file creations/deletions category table</summary>

|value|meaning|
|-|-|
|0|0|
|1|1-5|
|2|6-10|
|3|>10|

</details>

<details> <summary> Service operating state category table</summary>

|operating_state|label|
|--|--|
|0|UNUSED|
|1|RUNNING|
|2|STOPPED|
|3|PAUSED|
|4|DISABLED|
|5|INSTALLING|
|6|RESTARTING|

</details>

<details> <summary> Application operating state category table</summary>

|operating_state|label|
|--|--|
|0|UNUSED|
|1|RUNNING|
|2|CLOSED|
|3|INSTALLING|

</details>

<details> <summary> Service/Application health state category table</summary>

|health_state|label|
|--|--|
|0|UNUSED|
|1|GOOD|
|2|FIXING|
|3|COMPROMISED|
|4|OVERWHELMED|

</details>

<details> <summary> Application number of executions category table</summary>

|num_executions label|meaning|
|-|-|
|0|0|
|1|1-5|
|2|6-10|
|3|>10|

</details>

<details> <summary> Folder/file health status table</summary>

|health_status label|meaning|
|-|-|
|0|UNUSED|
|1|GOOD|
|2|COMPROMISED|
|3|CORRUPT|
|4|RESTORING|
|5|REPAIRING|

</details>

<details> <summary> File number of access category table</summary>

|num_access label|meaning|
|-|-|
|0|0|
|1|1-5|
|2|6-10|
|3|>10|

</details>

<details> <summary> NICs' operating_status category table</summary>

|operating_status|label|
|--|--|
|0|UNUSED|
|1|ENABLED|
|2|DISABLED|

</details>

<details> <summary> NIC monitored traffic utilisation category table </summary>

|load observation|percent utilisation|
|--|--|
|0|exactly 0%|
|1|0-11%|
|2|11-22%|
|3|22-33%|
|4|33-44%|
|5|44-55%|
|6|55-66%|
|7|66-77%|
|8|77-88%|
|9|88-99%|
|10|exactly 100%|


</details>



#### Routers and Firewalls

In addition, the agent can observe the list of Access Control List rules present on routers and firewalls.

`Routers` have one ACL which apply to each network interface (which in the context of networking devices are referenced as ports).

`Firewalls` have six ACLs and three ports. These port are predefined as the `Internal`, `External` and `DMZ` port. Each port comes with two ACL lists - inbound and outbound which apply to traffic ingress and egress on a specific port.

by default, the UC7 agent is configured to observe `3` different routers:

- `ST_INTRA-PRV-RT-CR`
- `ST_INTRA-PRV-RT-DR-1`
- `REM-PUB-RT-DR`


<details> <summary> Port operating status category table</summary>

|operating_status|label|
|--|--|
|0|UNUSED|
|1|ENABLED|
|2|DISABLED|

</details>

<details> <summary> ACL Permission category table</summary>

|permission|label|
|--|--|
|0|UNUSED|
|1|ALLOW|
|2|DENY|

</details>


<details> <summary> ACL source/destination IP ID category table</summary>

|IP ID|IP address| Associated node|
|--|--|--|
|0|UNUSED|
|1|ALL IPs|
|2|192.168.1.2 | HOME-PUB-PC-1     |
|3|192.168.1.3 | HOME-PUB-PC-2     |
|4|192.168.1.4 | HOME-PUB-PC-SRV   |
|5|192.168.20.2 |  REM-PUB-PC-1    |
|6|192.168.20.3 |  REM-PUB-PC-2    |
|7|192.168.20.4 |  REM-PUB-SRV     |
|8|192.168.100.2| ST_PUB_SRV_WEB_IP |
|9|192.168.200.2 | ST_HO-PRV-PC-1  |
|10|192.168.200.3 | ST_HO-PRV-PC-2  |
|11|192.168.200.4 | ST_HO-PRV-PC-3  |
|12|192.168.210.2 | ST_HR-PRV-PC-1  |
|13|192.168.210.3 | ST_HR-PRV-PC-2  |
|14|192.168.210.4 | ST_HR-PRV-PC-3  |
|15|192.168.220.2 | ST_DATA-PRV-SRV-STORAGE | 
|16|192.168.220.3 | ST_DATA-PRV-SRV-DB    |
|17|192.168.230.2 | PROJ-A-PRV-PC-1 |
|18|192.168.230.3 | PROJ-A-PRV-PC-2 |
|19|192.168.230.4 | PROJ-A-PRV-PC-3 |
|20|192.168.240.2 | PROJ-B-PRV-PC-1 |
|21|192.168.240.3 | PROJ-B-PRV-PC-2 |
|22|192.168.240.4 | PROJ-B-PRV-PC-3 |
|23|192.168.250.2 | PROJ-C-PRV-PC-1 |
|24|192.168.250.3 | PROJ-C-PRV-PC-2 |
|25|192.168.250.4 | PROJ-C-PRV-PC-3 |

</details>

<details> <summary> ACL Rule IP Address Wildcard category table</summary>

|wildcard|label|
|--|--|
|0|UNUSED|
|1|None|
|2|0.0.0.1|
|3|0.0.0.255|
|4|0.0.255.255|
</details>

<details> <summary> ACL Rule Port category table</summary>

|permission|label|used for|
|--|--|--|
|0|UNUSED|padding|
|1|ANY||
|2|21|FTP|
|3|53|DNS|
|4|80|HTTP|
|5|123|NTP|
|6|5432|POSTGRES SERVER|
|7|22|SSH|

</details>

<details> <summary> ACL Rule Protocol category table</summary>

|permission|label|
|--|--|
|0|UNUSED|
|1|ANY|
|2|ICMP|
|3|TCP|
|4|UDP|

</details>

<details> <summary> Firewall ports </summary>

|index|label|
|--|--|
|1|external|
|2|internal|
|3|DMZ|

</details>


In [ ]:
obs, reward, _,_,info = env.step(0)
for node_id, node_obs in obs['NODES'].items():
    if not "HOST" in node_id: # filter out hosts OBS and focus on ROUTERs
        print(node_id)
        pprint(node_obs)


ROUTER0
{'ACL': {0: {'dest_ip_id': 0,
             'dest_port_id': 0,
             'dest_wildcard_id': 0,
             'permission': 0,
             'position': 0,
             'protocol_id': 0,
             'source_ip_id': 0,
             'source_port_id': 0,
             'source_wildcard_id': 0},
         1: {'dest_ip_id': 0,
             'dest_port_id': 0,
             'dest_wildcard_id': 0,
             'permission': 0,
             'position': 1,
             'protocol_id': 0,
             'source_ip_id': 0,
             'source_port_id': 0,
             'source_wildcard_id': 0},
         2: {'dest_ip_id': 0,
             'dest_port_id': 0,
             'dest_wildcard_id': 0,
             'permission': 0,
             'position': 2,
             'protocol_id': 0,
             'source_ip_id': 0,
             'source_port_id': 0,
             'source_wildcard_id': 0},
         3: {'dest_ip_id': 0,
             'dest_port_id': 0,
             'dest_wildcard_id': 0,
             'perm

## AGENTS | Blue Agent | Action Space

***

#### `action_map`

Numerically ordered, the `action_map` define the actual details of what actions and the amount of actions that a blue agent can perform.

For example, the snippet below details the first four actions the the default UC7 blue agent is setup with:

```yaml
    action_space:
      action_map:
        0:
          action: do-nothing
          options: {}

        # |======================================|
        # |          ST_PROJ-A-PRV-PC-1          |
        # |======================================|

        # ST_PROJ-A-PRV-PC-1 | node-os-scan
        1:
          action: node-os-scan
          options:
            node_name: ST_PROJ-A-PRV-PC-1
        # ST_PROJ-A-PRV-PC-1 | node-shutdown
        2:
          action: node-shutdown
          options:
            node_name: ST_PROJ-A-PRV-PC-1
        # ST_PROJ-A-PRV-PC-1 | node-startup
        3:
          action: node-startup
          options:
            node_name: ST_PROJ-A-PRV-PC-1
```

Converting the yaml snippet below we end up with the following:

|Action Num | Action Type | Options|
|:---------:|:-----------:|:------:|
|0|**do-nothing**|*n/a*|
|1|**node-os-scan**|*node_name: ST_PROJ-A-PRV-PC-1*|
|2|**node-shutdown**|*node_name: ST_PROJ-A-PRV-PC-1*|
|3|**node-startup**|*node_name: ST_PROJ-A-PRV-PC-1*|


`0: do-nothing`:

The first action, `do-nothing` is the default standard that all agents are setup to use by default in primAITE. Quite simply this action makes no impact to the simulation - literally does nothing. Although this obviously does not seem all that useful, in practice an agent with a small yet impactful actions (such as adding or removing ACL's rules) may find that performing no action may be better than using a potentially detrimental one .

Additionally, you may spotted the code snippet below dotted around this notebook and many others.

```py
  env.step(0)
```

This code snippet is used to step forward in an PrimAITE episode and force the blue agent into performing no action which is very useful for demonstrating default simulation behaviour as well as the different impacts that the green and red agents have upon the environment.

In [ ]:
env.reset()
env.step(0)
defender = env.game.rl_agents.get("defender")
defender.show_history(ignored_actions=['']) # By default `show_history()` will ignore 'do-nothing'

2025-03-14 15:51:43,512: Resetting environment, episode 4, avg. reward: 2.225


Actions for 'defender':
+------+------------+--------+----------+---------------+
| Step |   Action   | Params | Response | Response Data |
+------+------------+--------+----------+---------------+
|  0   | do-nothing |        | success  |               |
+------+------------+--------+----------+---------------+


`1: node-os-scan`:

The first actual action that the blue agent can perform is scanning action. The blue agent is capable of a variety of different scanning type actions (such as `node-application-scan` or `node-file/folder-scan`) which can be used to gain a deeper understanding of the simulation state. Specifically, these actions will cause the blue agent's observations to update to the **"true"** `health_status` of a simulation component. The `node-os-scan` acts a combined version of all these scan type actions.

For example, if a red agent corrupts and alters the health status of a file, the blue agent's observation space will not reflect this until the agent performs a `node-file-scan` on the newly corrupted file. It's worth noting that blue agents can be configured to see the true `health_status` of software and files without needing to scan in the yaml. Although this may make it easier for an train and create an effective blue agent it could be seen as reducing the fidelity of the simulation.

The code snippet below demonstrates an example where the blue agent uses the `node-os-scan` action to reveal the true health status `ST_PROJ-A-PRV-PC-1`.

In [ ]:
obs, reward, term, trunc, info = env.step(0)
print(f"ftp-client (Prior Scan) OBS: {defender.observation_manager.current_observation['NODES']['HOST0']['SERVICES'][1]}")
print(f"database-client (Prior Scan) OBS: {defender.observation_manager.current_observation['NODES']['HOST0']['APPLICATIONS'][2]}")

ftp-client (Prior Scan) OBS: {'operating_status': 2, 'health_status': 0}
database-client (Prior Scan) OBS: {'operating_status': 1, 'health_status': 0, 'num_executions': 1}


In [ ]:
st_project_a_private_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("ST_PROJ-A-PRV-PC-1")
st_project_a_private_pc_1.software_manager.software["ftp-client"].set_health_state(SoftwareHealthState.COMPROMISED)
st_project_a_private_pc_1.software_manager.software["database-client"].set_health_state(SoftwareHealthState.COMPROMISED)
st_project_a_private_pc_1.software_manager.show()

+---------------------------------------------------------------------------------------+
|                          ST_PROJ-A-PRV-PC-1 Software Manager                          |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

Set by the `node_scan_duration` option in the simulation `defaults` section, the results of `node-os-scan` take **8** timesteps before it impacts the blue agent's observation space.

In [ ]:
print(f'Node OS Scan time step duration: {cfg["simulation"]["defaults"]["node_scan_duration"]}')
env.step(1)
print(defender.show_history())
for _ in range(9):
    obs, reward, term, trunc, info = env.step(0)

print(f"Current Simulation Time Step: {env.game.step_counter}")
print(f"ftp-client (Post Scan) OBS: {defender.observation_manager.current_observation['NODES']['HOST0']['SERVICES'][1]}")
print(f"database-client (Post Scan) OBS: {defender.observation_manager.current_observation['NODES']['HOST0']['APPLICATIONS'][2]}")

Node OS Scan time step duration: 8
Actions for 'defender':
+------+--------------+-------------------------------+----------+---------------+
| Step |    Action    |             Params            | Response | Response Data |
+------+--------------+-------------------------------+----------+---------------+
|  2   | node-os-scan | node_name: ST_PROJ-A-PRV-PC-1 | success  |               |
|      |              |                               |          |               |
+------+--------------+-------------------------------+----------+---------------+
None
Current Simulation Time Step: 12
ftp-client (Post Scan) OBS: {'operating_status': 2, 'health_status': 3}
database-client (Post Scan) OBS: {'operating_status': 1, 'health_status': 3, 'num_executions': 1}


`2: node-shutdown`:

The next action available is `node-shutdown`. This action quite simply attempts to shut down the given `node_name` which in this case is set to `ST_PROJ-A-PRV-PC-1`. Shutting a PC down affects the `operating_status` of the host machine which the following snippets demonstrate.

In [ ]:
# `1` is equal to 'ON' in this case.
obs, reward, term, trunc, info = env.step(0)
print(f"ST_PROJ-A-PRV-PC-1's (prior `node-shutdown`) operating state: {defender.observation_manager.current_observation['NODES']['HOST0']['operating_status']}")

ST_PROJ-A-PRV-PC-1's (prior `node-shutdown`) operating state: 1


As mentioned previously, some actions require a number of timesteps to elapse before their impact is resolved within the simulation. `node-shutdown` by default takes three timesteps to take effect.

In [ ]:
obs, reward, term, trunc, info = env.step(2)
# Skipping three timesteps by forcing the blue agent into performing a `do-nothing` action.
for _ in range(3):
    env.step(0) 

In [ ]:
obs, reward, term, trunc, info = env.step(0)
print(f"ST_PROJ-A-PRV-PC-1's (post `node-shutdown`) operating state: {defender.observation_manager.current_observation['NODES']['HOST0']['operating_status']}")

ST_PROJ-A-PRV-PC-1's (post `node-shutdown`) operating state: 2


`3: node-startup`:

Lastly, the blue agent third action `node-startup` can be used to bring the `ST_PROJ-A-PRV-PC-1` back up and running. Similar to the previous action, `node-startup` takes three timesteps.


In [ ]:
obs, reward, term, trunc, info = env.step(3)

for _ in range(3):
    env.step(0) # 3 second reboot time.
    
obs, reward, term, trunc, info = env.step(0)
print(f"ST_PROJ-A-PRV-PC-1's (post `node-startup`) operating state: {defender.observation_manager.current_observation['NODES']['HOST0']['operating_status']}")

ST_PROJ-A-PRV-PC-1's (post `node-startup`) operating state: 1


In [ ]:
print(defender.show_history())

Actions for 'defender':
+------+---------------+-------------------------------+----------+---------------+
| Step |     Action    |             Params            | Response | Response Data |
+------+---------------+-------------------------------+----------+---------------+
|  2   |  node-os-scan | node_name: ST_PROJ-A-PRV-PC-1 | success  |               |
|      |               |                               |          |               |
|  13  | node-shutdown | node_name: ST_PROJ-A-PRV-PC-1 | success  |               |
|      |               |                               |          |               |
|  18  |  node-startup | node_name: ST_PROJ-A-PRV-PC-1 | success  |               |
|      |               |                               |          |               |
+------+---------------+-------------------------------+----------+---------------+
None


## AGENTS | Blue Agent | Reward Components

In order to add context to the observation, the blue agent can be configured with different reward components. For any readers unfamiliar with PrimAITE rewards then user guide contains a good introduction as this notebook section will only cover the UC7 blue agent setup.

For `UC7`, the blue agent is setup to receive a negative reward when the `database.db` file integrity is affected (i.e enters into a `COMPROMISED` state):

```yaml
    reward_function:
      reward_components:
        - type: database-file-integrity
          weight: *HIGH_WEIGHT_IMPACT # Equal to 0.95 (Reward Anchors defined at lines 960 - 980 in the uc7_config.yaml)
          options: 
            node_hostname: ST_DATA-PRV-SRV-DB 
            folder_name: database
            file_name: database.db
```

The blue agent's remaining reward function is comprised of **32** different ``shared-reward`` components. These rewards will grant the blue agent a positive or negative reward based on the current reward of the **32** green agents. 

In [ ]:
table = PrettyTable()
table.field_names = ["Reward Type", "Reward Option", "Reward Weight"]
for i in range(len(defender.reward_function.reward_components)):
    reward_type = defender.reward_function.reward_components[i][0].config.type
    try:
        reward_option = defender.reward_function.reward_components[i][0].config.file_name
    except:
        reward_option = defender.reward_function.reward_components[i][0].config.agent_name
    reward_weight = defender.reward_function.reward_components[i][1]
    table.add_row(row=[reward_type, reward_option, reward_weight])
print(table)


+-------------------------+-------------------------+---------------+
|       Reward Type       |      Reward Option      | Reward Weight |
+-------------------------+-------------------------+---------------+
| database-file-integrity |       database.db       |      0.95     |
|      shared-reward      |     HOME_WORKER-1-DB    |    0.03125    |
|      shared-reward      |    HOME_WORKER-1-WEB    |    0.03125    |
|      shared-reward      |     HOME_WORKER-2-DB    |    0.03125    |
|      shared-reward      |    HOME_WORKER-2-WEB    |    0.03125    |
|      shared-reward      |    REMOTE_WORKER-1-DB   |    0.03125    |
|      shared-reward      |   REMOTE_WORKER-1-WEB   |    0.03125    |
|      shared-reward      |    REMOTE_WORKER-2-DB   |    0.03125    |
|      shared-reward      |   REMOTE_WORKER-2-WEB   |    0.03125    |
|      shared-reward      |   PROJ_A-SENIOR-DEV-DB  |    0.03125    |
|      shared-reward      |  PROJ_A-SENIOR-DEV-WEB  |    0.03125    |
|      shared-reward

By default, each of the `shared-reward` component is configured with a equal reward `weight` of `0.03125` which totals a blue agent reward weight of `1`. 

It's worth noting that `shared-reward` components are **not** required to have a equal weight or total a weight value under `1`. 

Users are recommended to alter the `weights` of these rewards when creating their own scenarios.

```yaml

# UC7 Shared Reward Component Green Agents (32 Green Agents each contributing 0.03125 of blue reward)

# Blue Shared Reward | HOME_WORKER-1-DB
- type: shared-reward
    weight: 0.03125
    options:
    agent_name: HOME_WORKER-1-DB

# Green Agent HOME_WORKER-1-DB's reward function:
    reward_function:
      reward_components:
      - type: green-admin-database-unreachable-penalty
        weight: *MEDIUM_WEIGHT_IMPACT # Equal to 0.5 (Reward Anchors defined at lines 960 - 980 in the uc7_config.yaml)
        options:
          node_hostname: HOME-PUB-PC-1

```

The `weight` option in a `shared-reward` reward acts a multiplier to the reward of agent given in `agent_name`:

shared_reward = agent_reward x shared_reward_weight


This can be a little difficult to understand intuitively so the following code snippets demonstrate how one of these rewards are calculated during a live episode.

In [ ]:
# Readers running this notebook natively can use edit this to test out different reward weight combinations
BLUE_AGENT_SHARED_REWARD_WEIGHT = 5

For example, if a user wished to configure the blue agent to place more value on the head office green agents such as the `CEO` then the blue agent's `shared-reward` components could be altered to reflect this by increasing the `weight` of the `shared-reward` configured to the `CEO` green agent.

In [ ]:
# Reloads the UC7 config and removes all of other reward-components. 
BLUE_AGENT_INDEX = 33
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)

    # Removing all the other blue agent rewards and adding a custom blue reward
    blue_shared_reward_ceo = {'type': 'shared-reward', 'weight': BLUE_AGENT_SHARED_REWARD_WEIGHT, 'options': {'agent_name': 'CEO'}}

    # Add the new custom blue agent shared rewards
    blue_shared_reward_home_worker = cfg['agents'][BLUE_AGENT_INDEX]['reward_function']['reward_components'].pop(1)
    cfg['agents'][BLUE_AGENT_INDEX]['reward_function']['reward_components'].clear() # Remove all blue agent rewards
    cfg['agents'][BLUE_AGENT_INDEX]['reward_function']['reward_components'].append(blue_shared_reward_ceo) 
    cfg['agents'][BLUE_AGENT_INDEX]['reward_function']['reward_components'].append(blue_shared_reward_home_worker) 


env = PrimaiteGymEnv(env_config=cfg)
env.reset()

# Run the episode 10 times and record the results
table = PrettyTable()
table.field_names = ["Time Step", "Home Worker Reward", "CEO Reward", "Blue Agent Total Reward"]
for _ in range(10):
    env.step(0)
    home_worker = env.game.agents.get('HOME_WORKER-1-DB')
    ceo = env.game.agents.get('CEO')
    defender = env.game.agents.get("defender")
    table.add_row([env.game.step_counter,home_worker.reward_function.current_reward, ceo.reward_function.current_reward, defender.reward_function.current_reward])
print(table)


2025-03-14 15:51:47,022: PrimaiteGymEnv RNG seed = None
2025-03-14 15:51:47,025: Resetting environment, episode 0, avg. reward: 0.0


+-----------+--------------------+------------+-------------------------+
| Time Step | Home Worker Reward | CEO Reward | Blue Agent Total Reward |
+-----------+--------------------+------------+-------------------------+
|     1     |        0.0         |    0.0     |           0.0           |
|     2     |        0.0         |    0.95    |           4.75          |
|     3     |        0.0         |    0.95    |           4.75          |
|     4     |        0.0         |    0.95    |           4.75          |
|     5     |        0.5         |    0.95    |         4.765625        |
|     6     |        0.5         |    0.95    |         4.765625        |
|     7     |        0.5         |    0.95    |         4.765625        |
|     8     |        0.5         |    0.95    |         4.765625        |
|     9     |        0.5         |    0.95    |         4.765625        |
|     10    |        0.5         |    0.95    |         4.765625        |
+-----------+--------------------+----

As you can see from the table above, because we increased the `shared-reward` weightings the blue agent's reward is nearly all comprised of the CEO's reward - `4.75`:

ceo_reward_contribution = 0.95 x 5

We can see that the remote worker agent only contributes `0.015625` to the blue agent's total reward:

remote_work_reward_contribution = 0.5 x 0.03125


Lastly, the final few code snippets demonstrate how the default UC7 blue agent's reward is affected by simulation state within an episode.

In a perfect episode, with no red agent interference, the blue agent can expect the following reward:

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg["agents"].pop(32) # removing red agent
env = PrimaiteGymEnv(env_config=cfg)
env.reset()
defender = env.game.rl_agents.get("defender")
for _ in range(128):
    env.step(0)
defender.reward_function.total_reward

2025-03-14 15:51:49,242: PrimaiteGymEnv RNG seed = None
2025-03-14 15:51:49,244: Resetting environment, episode 0, avg. reward: 0.0


170.8687499999996

The below code snippet shows what you'd expect the defender's total reward to be after TAP001 is left unchallenged:

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
env = PrimaiteGymEnv(env_config=cfg)
env.reset()
defender = env.game.rl_agents.get("defender")
for _ in range(128):
    env.step(0)
print(f"Successful TAP001 & Blue Agent Reward: {defender.reward_function.total_reward}")

2025-03-14 15:51:57,812: PrimaiteGymEnv RNG seed = None
2025-03-14 15:51:57,816: Resetting environment, episode 0, avg. reward: 0.0


Successful TAP001 & Blue Agent Reward: 140.27031249999996


Next is what you'd expect the defender's total reward to be after TAP003 is left unchallenged:

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config_tap003.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg["agents"][33]["agent_settings"]["flatten_obs"] = False
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['io_settings']['save_agent_actions'] = True # Saving syslogs
    cfg['io_settings']['save_agent_logs'] = True # Saving agent logs
env = PrimaiteGymEnv(env_config=cfg)
env.reset()
defender = env.game.rl_agents.get("defender")
for _ in range(128):
    env.step(0)
print(f"Successful TAP003 & Blue Agent Reward: {defender.reward_function.total_reward}")

2025-03-14 15:52:10,712: PrimaiteGymEnv RNG seed = None
2025-03-14 15:52:10,715: Resetting environment, episode 0, avg. reward: 0.0
2025-03-14 15:52:10,719: Saving agent action log to C:\Users\CharlieCrane\primaite\4.0.0-dev\sessions\2025-03-14\15-51-12\agent_actions\episode_0.json


Successful TAP003 & Blue Agent Reward: 116.40624999999962


Lastly, if we completely block any green traffic and set the database.db to `COMPROMISED` we can simulate the potential worst case situation for the blue agent (based off the default `reward_components`).

In [ ]:
env = PrimaiteGymEnv(env_config=cfg)
env.reset()
defender = env.game.rl_agents.get("defender")

# Corrupting and Disabling the database-service
st_data_private_server_database: Server = env.game.simulation.network.get_node_by_hostname("ST_DATA-PRV-SRV-DB")
st_data_private_server_database_file = st_data_private_server_database.file_system.get_file(folder_name="database", file_name="database.db")
st_data_private_server_database_file.health_status = FileSystemItemHealthStatus.COMPROMISED
st_data_private_server_database.software_manager.software["database-service"].operating_state = ServiceOperatingState.DISABLED

# Shutting down the web-server

st_dmz_pub_srv_web: Server = env.game.simulation.network.get_node_by_hostname("ST_DMZ-PUB-SRV-WEB")
st_dmz_pub_srv_web.software_manager.software["web-server"].operating_state = ServiceOperatingState.DISABLED

# Shutting down the dns-server

isp_pub_srv_dns_server: Server = env.game.simulation.network.get_node_by_hostname("ISP-PUB-SRV-DNS")
isp_pub_srv_dns_server.software_manager.software["dns-server"].operating_state = ServiceOperatingState.DISABLED

for _ in range(128):
    env.step(0)
print(f"Worst Case Episode Blue Agent Reward: {defender.reward_function.total_reward}")

2025-03-14 15:52:17,859: PrimaiteGymEnv RNG seed = None
2025-03-14 15:52:17,860: Resetting environment, episode 0, avg. reward: 0.0
2025-03-14 15:52:17,863: Saving agent action log to C:\Users\CharlieCrane\primaite\4.0.0-dev\sessions\2025-03-14\15-51-12\agent_actions\episode_0.json


Worst Case Episode Blue Agent Reward: -170.5046874999996
